# Explorações iniciais

## 0. Setup

In [1]:
#---- Bibliotecas:

# Análise e manipulação de dados:

import pandas as pd
import numpy as np

# Visualização de dados:

import plotly.io as pio
import plotly.express as px

In [2]:
#---- Data path:

data_path = '../../03-data/'

#---- Template dos gráficos:

pio.templates.default = "plotly_white"

## 1. Extração dos dados

### 1.1. Dimensão de livros

In [3]:
books = pd.read_csv(data_path + '01-raw/Books.csv', low_memory = False)

books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### 1.2. Dimensão de Usuários

In [4]:
users = pd.read_csv(data_path + '01-raw/Users.csv', low_memory = False)

users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


### 1.3. Fato de avaliações dos livros pelos usuários

In [5]:
ratings = pd.read_csv(data_path + '01-raw/Ratings.csv', low_memory = False)

ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


## 2. Manuseio dos dados

### 2.1. Abrindo a variável de Location

In [6]:
users[['city', 'state', 'country']] = users['Location'].str.split(pat = ',', n = 2, expand = True)

users.head()

,User-ID,Location,Age,city,state,country
0,1,"nyc, new york, usa",NaN,nyc,new york,usa
1,2,"stockton, california, usa",18.0,stockton,california,usa
2,3,"moscow, yukon territory, russia",NaN,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17.0,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",NaN,farnborough,hants,united kingdom


### 2.2. Join nos dados

In [7]:
df_ratings_user = ratings\
    .merge(books, on = 'ISBN', how = 'left')\
    .merge(users, on = 'User-ID', how = 'left')\
    .drop(columns = ['Location'] + [col for col in books.columns if 'URL' in col])

df_ratings_user.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,NaN,tyler,texas,usa
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,NaN,seattle,washington,usa
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,16.0,h,new south wales,australia
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,16.0,rijeka,n/a,croatia
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,16.0,rijeka,n/a,croatia


### 2.3. Split em treino e teste

**Regra:** 

- Filtra avaliações diferentes de 0
- Filtrar usuários que tem 3 ou mais avaliações
- Treino: Todas as avaliações do usuário, exceto a última
- Teste: Última avaliação do usuário

In [8]:
#---- Filtrar avaliações diferentes de 0

df_ratings_user_filtered = df_ratings_user\
    .copy()\
    .query('`Book-Rating` != 0')

df_ratings_user_filtered.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,NaN,seattle,washington,usa
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,16.0,rijeka,n/a,croatia
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,16.0,rijeka,n/a,croatia
6,276736,3257224281,8,NaN,NaN,NaN,NaN,NaN,salzburg,salzburg,austria
7,276737,0600570967,6,NaN,NaN,NaN,NaN,14.0,sydney,new south wales,australia


In [9]:
#---- Filtrar usuários que tem 3 ou mais avaliações

# Criando a variável de RowNumber

df_ratings_user_filtered['RN'] = df_ratings_user_filtered\
    .groupby('User-ID')\
    .cumcount() + 1

In [10]:
# Separando os usuários que tem 3 ou mais avaliações diferentes de 0

users_3more_rating = df_ratings_user_filtered\
    .groupby('User-ID', as_index = False)['RN']\
    .max()\
    .query('RN >= 3')['User-ID']\
    .to_list()

In [11]:
# Filtrando os usuários acima:

df_ratings_user_filtered = df_ratings_user_filtered\
    .query(f'`User-ID`.isin({users_3more_rating})')

df_ratings_user_filtered.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN
16,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,25.0,iowa city,iowa,usa,1
19,276747,0671537458,9,Waiting to Exhale,Terry McMillan,1995,Pocket,25.0,iowa city,iowa,usa,2
20,276747,0679776818,8,Birdsong: A Novel of Love and War,Sebastian Faulks,1997,Vintage Books USA,25.0,iowa city,iowa,usa,3
21,276747,0943066433,7,How to Deal With Difficult People,Rick Brinkman,1995,Careertrack Inc.,25.0,iowa city,iowa,usa,4
23,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5


In [12]:
# Separando os dados de teste:

df_last_item_user_id = df_ratings_user_filtered\
    .groupby('User-ID', as_index = False)['RN']\
    .max()

df_test = df_ratings_user_filtered\
    .merge(df_last_item_user_id, on = ['User-ID', 'RN'], how = 'inner')

df_test

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN
0,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5
1,276762,N3453124715,4,NaN,NaN,NaN,NaN,25.0,duisburg,nordrhein-westfalen,germany,4
2,276772,3596151465,10,Henry der Held.,Roddy Doyle,2001,"Fischer (Tb.), Frankfurt",NaN,bonn,nordrhein-westfalen,germany,3
3,276786,8478442588,6,El Elogio de La Sombra,Tanazaki,1998,Siruela,34.0,madrid,madrid,spain,4
4,276788,055310666X,10,False Memory,Dean R. Koontz,1999,Bantam Books,NaN,mentone,california,usa,3
...,...,...,...,...,...,...,...,...,...,...,...,...
22563,276680,1931561648,9,The Time Traveler's Wife (Today Show Book Club...,Audrey Niffenegger,2003,MacAdam/Cage Publishing,55.0,hopewell junction,new york,usa,91
22564,276681,1931333246,10,Chorus,Kurt Messick,2003,Dry Bones Press,43.0,chicago,illinois,usa,11
22565,276683,0553278231,9,Cannery Row,John Steinbeck,1986,Bantam Books,NaN,charlotte,north carolina,usa,3
22566,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,NaN,fort lee,new jersey,usa,29


In [13]:
#---- Dados de treino

df_train = df_ratings_user_filtered\
    .merge(df_last_item_user_id.assign(indicador = 1), on = ['User-ID', 'RN'], how = 'left')\
    .query('indicador.isna()')\
    .drop(columns = 'indicador')

df_train

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN
0,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,25.0,iowa city,iowa,usa,1
1,276747,0671537458,9,Waiting to Exhale,Terry McMillan,1995,Pocket,25.0,iowa city,iowa,usa,2
2,276747,0679776818,8,Birdsong: A Novel of Love and War,Sebastian Faulks,1997,Vintage Books USA,25.0,iowa city,iowa,usa,3
3,276747,0943066433,7,How to Deal With Difficult People,Rick Brinkman,1995,Careertrack Inc.,25.0,iowa city,iowa,usa,4
5,276762,0380711524,5,See Jane Run,Joy Fielding,1992,Avon,25.0,duisburg,nordrhein-westfalen,germany,1
...,...,...,...,...,...,...,...,...,...,...,...,...
368572,276688,1551669315,6,The Third Wife,Jasmine Cresswell,2002,Mira,NaN,fort lee,new jersey,usa,28
368574,276704,0345386108,6,Winter Moon,Dean R. Koontz,1995,Ballantine Books,NaN,cedar park,texas,usa,1
368575,276704,0380796155,5,White Abacus,Damien Broderick,1998,Eos,NaN,cedar park,texas,usa,2
368576,276704,0743211383,7,Dreamcatcher,Stephen King,2001,Scribner,NaN,cedar park,texas,usa,3


### 2.4. Escolha dos top 5, 10 e 20 livros 

**Regras**: 

- Melhores avaliações (média)
- Ter sido avaliado por 10 ou mais usuários

In [14]:
top20_books = df_ratings_user_filtered\
    .groupby('Book-Title', as_index = False)\
    .agg(
        media_rating = ('Book-Rating', 'mean'),
        frequencia = ('ISBN', 'count')
    )\
    .query('frequencia > 10')\
    .sort_values(by = 'media_rating', ascending = False)\
    .head(20)['Book-Title']\
    .to_list()

top20_books

['Postmarked Yesteryear: 30 Rare Holiday Postcards',
 'Dilbert: A Book of Postcards',
 'Harry Potter and the Chamber of Secrets Postcard Book',
 'Route 66 Postcards: Greetings from the Mother Road',
 'Fox in Socks (I Can Read It All by Myself Beginner Books)',
 'The Giving Tree',
 'The Return of the King (The Lord of The Rings, Part 3)',
 'The Authoritative Calvin and Hobbes (Calvin and Hobbes)',
 "My Sister's Keeper : A Novel (Picoult, Jodi)",
 'Where the Sidewalk Ends : Poems and Drawings',
 'Calvin and Hobbes',
 'Maus a Survivors Tale: My Father Bleeds History',
 'Are You My Mother?',
 '52 Deck Series: 52 Ways to Celebrate Friendship',
 'Mrs. Mike',
 "The Lion, the Witch and the Wardrobe (Full-Color Collector's Edition)",
 "Oh, the Places You'll Go!",
 'Die unendliche Geschichte: Von A bis Z',
 'Complete Chronicles of Narnia',
 "Scientific Progress Goes 'Boink':  A Calvin and Hobbes Collection"]

## 3. Exportação dos dados

In [22]:
#---- Treino: 

df_train\
        .to_parquet(path = data_path + '02-processed/01-train_data.parquet', index = False)

In [21]:
#---- Teste: 

df_test\
        .to_parquet(path = data_path + '02-processed/01-test_data.parquet', index = False)

In [20]:
#---- Lista com os top20 livros

import pickle
with open(data_path + '02-processed/03-top20-books.pkl', 'wb') as fp:
    pickle.dump(top20_books, fp)